In [ ]:
from google.colab import drive

drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# pip install tflearn

In [ ]:
import json
import numpy as np
import tensorflow as tf
import nltk
# import tensorflow as tf
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense, Activation, Dropout
# from tensorflow.keras.optimizers import SGD
json_path = '/content/drive/MyDrive/chatbot/intents.json'

In [ ]:
with open(json_path) as intention:
    intents = json.load(intention)
# print(intents)

In [ ]:
words = []
tag = []
docs = []
ignore=['!','?','.....',',','.']

In [ ]:
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   U

True

In [ ]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        token = nltk.word_tokenize(pattern)
        words.extend(token)
        docs.append((token,intent['tag']))
        if intent['tag'] not in tag:
            tag.append(intent['tag'])



In [ ]:
print(words)
print(tag)
print(docs)

['Hi', 'Halo', '!', 'Ada', 'orang', 'kah', '?', 'Salam', 'kenal', '!', 'Halo', ',', 'aku', '.....', 'P', 'Assalamualaikum', 'Bye', 'Dadah', 'Sampai', 'jumpa', '!', 'See', 'you', 'later', 'Dah', 'Thanks', 'Thank', 'you', 'Terima', 'kasih', '!', 'Makasih', 'Ada', 'apa', 'sih', 'jenis', 'kopi', '?', 'Jenis', 'kopi', 'di', 'Indonesia', 'ada', 'apa', 'aja', '?', 'Jenis', 'kopi', 'ada', 'apa', 'aja', '?', 'Bagaimana', 'cara', 'brew', 'kopi', '?', 'Bagaimana', 'teknik', 'brew', 'kopi', '?', 'Bagaimana', 'cara', 'membuat', 'kopi', 'yang', 'baik', '?', 'Apakah', 'kopi', 'punya', 'rasa-rasa', 'lain', '?', 'Beritahu', 'saya', 'tentang', 'rasa', 'kopi', 'Rasa', 'apa', 'saja', 'yang', 'bisa', 'saya', 'temukan', 'di', 'kopi', '?', 'Berapa', 'tinggi', 'kafein', 'di', 'kopi', '?', 'Tingkatan', 'kafein', 'di', 'kopi', 'Apakah', 'kopi', 'memiliki', 'kafein', 'yang', 'tinggi', '?', 'Apa', 'brand', 'kopi', 'yang', 'terkenal', 'di', 'Indonesia', '?', 'Apa', 'brand', 'kopi', 'yang', 'disukai', 'di', 'Indone

In [ ]:
lemmatizer = nltk.WordNetLemmatizer()
words = [lemmatizer.lemmatize(token.lower()) for token in words if token not in ignore]
words = sorted(list(set(words)))

In [ ]:
wordss = np.array(words)
print(wordss.shape)

(59,)


In [ ]:
training_list = []
output_nan = [0] * len(tag)

for doc in docs:
    bow = []
    pattern = doc[0]
    pattern = [lemmatizer.lemmatize(sentence.lower()) for sentence in pattern]
    for word in words:
        bow.append(1) if word in pattern else bow.append(0)

    output = list(output_nan)
    output[tag.index(doc[1])] = 1
    bow_array = np.array(bow)
    output_array = np.array(output)

    training_list.append((bow_array, output_array))

training = np.array(training_list)
train_x = list(training[:,0])
train_y = list(training[:,1])

train_x = np.stack(train_x)
train_y = np.stack(train_y)

<ipython-input-11-5ef5f8e2f7c4>:18: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training_list)


In [ ]:
print(train_x.shape)

(31, 59)


In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(8, input_shape=(len(train_x[0]), )),
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dense(len(train_y[0]), activation='softmax')
])

model.compile(loss=tf.keras.losses.categorical_crossentropy, optimizer='adam', metrics=['acc'])
model.fit(train_x, train_y, epochs=1000, batch_size=8)

Epoch 1/1000
4/4 [==============================] - 2s 6ms/step - loss: 2.0227 - acc: 0.1935
Epoch 2/1000
4/4 [==============================] - 0s 4ms/step - loss: 2.0081 - acc: 0.1935
Epoch 3/1000
4/4 [==============================] - 0s 4ms/step - loss: 1.9941 - acc: 0.2903
Epoch 4/1000
4/4 [==============================] - 0s 5ms/step - loss: 1.9814 - acc: 0.2581
Epoch 5/1000
4/4 [==============================] - 0s 4ms/step - loss: 1.9670 - acc: 0.2581
Epoch 6/1000
4/4 [==============================] - 0s 5ms/step - loss: 1.9529 - acc: 0.2903
Epoch 7/1000
4/4 [==============================] - 0s 4ms/step - loss: 1.9390 - acc: 0.2903
Epoch 8/1000
4/4 [==============================] - 0s 4ms/step - loss: 1.9252 - acc: 0.2903
Epoch 9/1000
4/4 [==============================] - 0s 4ms/step - loss: 1.9098 - acc: 0.3226
Epoch 10/1000
4/4 [==============================] - 0s 4ms/step - loss: 1.8967 - acc: 0.3548
Epoch 11/1000
4/4 [==============================] - 0s 5ms/step - lo

In [ ]:
model.save('/content/drive/MyDrive/chatbot/model/chatbot_model_2.h5')

In [ ]:
load_model = tf.keras.models.load_model('/content/drive/MyDrive/chatbot/model/chatbot_model_2.h5')


In [ ]:
load_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 480       
                                                                 
 dense_1 (Dense)             (None, 8)                 72        
                                                                 
 dense_2 (Dense)             (None, 8)                 72        
                                                                 
Total params: 624
Trainable params: 624
Non-trainable params: 0
_________________________________________________________________


In [ ]:
with open('/content/drive/MyDrive/chatbot/intents.json') as f:
  intents = json.load(f)

In [ ]:
print(intents)

{'intents': [{'tag': 'greeting', 'patterns': ['Hi', 'Halo!', 'Ada orang kah?', 'Salam kenal!', 'Halo, aku .....', 'P', 'Assalamualaikum'], 'responses': ['Halo, saya Chatbot!', 'Haiiii! Adakah yang bisa saya bantu?', 'Halo, apa ada yang bisa saya bantu?']}, {'tag': 'goodbye', 'patterns': ['Bye', 'Dadah', 'Sampai jumpa!', 'See you later', 'Dah'], 'responses': ['See you later, terima kasih sudah mau mampir!', 'Byebyee!', 'Bye! Come back again soon yaa']}, {'tag': 'thanks', 'patterns': ['Thanks', 'Thank you', 'Terima kasih!', 'Makasih'], 'responses': ['Happy to help!', 'Any time!', 'My pleasure', 'Sama-sama, senang bisa membantu']}, {'tag': 'types', 'patterns': ['Ada apa sih jenis kopi?', 'Jenis kopi di Indonesia ada apa aja?', 'Jenis kopi ada apa aja?'], 'responses': ['Berdasarkan hal yang kamu tanyakan, saya menemukan tautan-tautan berikut yang bisa membantu', 'Hmmmm....ini adalah tautan-tautan yang saya kira bisa membantu kamu', 'Berdasarkan pertanyaan kamu, saya menemukan tautan-tautan

In [ ]:
words = []
classes = []
documents = []
ignore_words = ['?', '!','.....',',','.']

In [ ]:
for intent in intents['intents']:
  # print(intent)
  for p in intent['patterns']:
    print(p)
    w = nltk.word_tokenize(p)
    words.extend(w)
    documents.append((w, intent['tag']))
    if intent['tag'] not in classes:
      classes.append(intent['tag'])

Hi
Halo!
Ada orang kah?
Salam kenal!
Halo, aku .....
P
Assalamualaikum
Bye
Dadah
Sampai jumpa!
See you later
Dah
Thanks
Thank you
Terima kasih!
Makasih
Ada apa sih jenis kopi?
Jenis kopi di Indonesia ada apa aja?
Jenis kopi ada apa aja?
Bagaimana cara brew kopi?
Bagaimana teknik brew kopi?
Bagaimana cara membuat kopi yang baik?
Apakah kopi punya rasa-rasa lain?
Beritahu saya tentang rasa kopi
Rasa apa saja yang bisa saya temukan di kopi?
Berapa tinggi kafein di kopi?
Tingkatan kafein di kopi
Apakah kopi memiliki kafein yang tinggi?
Apa brand kopi yang terkenal di Indonesia?
Apa brand kopi yang disukai di Indonesia?
Sebutin dong beberapa brand kopi terkenal yang ada di Indonesia


In [ ]:
def clean_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words if word not in ignore_words]
    sentence_words = sorted(list(set(sentence_words)))
    return sentence_words

In [ ]:
def bag_of_words(sentence, words, show_details=False):
        sentence_words = clean_sentence(sentence)
        bag = [0] * len(words)
        for s in sentence_words:
            for i, w in enumerate(words):
                if w == s:
                    bag[i] = 1

        arrBag = np.array(bag)
        return np.stack(arrBag)

In [ ]:
import random
def generate_response(sentence):
        input_data = bag_of_words(sentence, words)
        input_data = input_data.reshape(1, input_data.shape[0])
        results = model.predict(input_data)[0]
        results_index = np.argmax(results)
        tag = classes[results_index]

        if results[results_index] > 0.7:
            for intent in intents['intents']:
                if intent['tag'] == tag:
                    response = random.choice(intent['responses'])
                    # links = intent.get('links', [])
                    return response
        else:
            return "I'm sorry, I didn't understand that.", []

In [ ]:
print(bag_of_words('P', words).shape)

(59,)


In [ ]:
print("Chatbot: Hello! How can I assist you?")
while True:
        user_input = input("User: ")
        if user_input.lower() == 'quit':
            print("Chatbot: Goodbye!")
            break
        response= generate_response(user_input)
        print("Chatbot:", response)

Chatbot: Hello! How can I assist you?
User: p
1/1 [==============================] - 0s 67ms/step
Chatbot: Halo, saya Chatbot!
User: assalamualaikum
1/1 [==============================] - 0s 26ms/step
Chatbot: Halo, saya Chatbot!
User: Jenis kopi di Indonesia ada apa aja?
1/1 [==============================] - 0s 41ms/step
Chatbot: Berdasarkan pertanyaan kamu, saya menemukan tautan-tautan berikut
User: Apa brand kopi yang terkenal di Indonesia?
1/1 [==============================] - 0s 24ms/step
Chatbot: Menarik. Saya mencoba mencari tahu beberapa brand kopi terkenal yang ada di Indonesia, mungkin kamu bisa cari tau beberapa brand yang tautannya ada di bawah.
User: Dadah
1/1 [==============================] - 0s 24ms/step
Chatbot: Bye! Come back again soon yaa


KeyboardInterrupt: ignored